### Using an external Rust library to speed up lon, lat to [BNG](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid) conversion

In [1]:
import numpy as np
import pandas as pd
import math
from ctypes import cdll, c_float, Structure, ARRAY, POINTER, c_int32, c_uint32, c_size_t, c_void_p, cast
from sys import platform
from bng import bng
import pyproj
import pdb

### Setting up the Rust library. See [here](https://github.com/alexcrichton/rust-ffi-examples/tree/master/python-to-rust) for more

Ensure you've built your Rust library using `cargo build --release`, or the next step will fail.

The boilerplate below can easily be hidden in a wrapper function – it's just here to demonstrate how to call into a shared Rust lib using FFI.

In [2]:
if platform == "darwin":
    ext = "dylib"
else:
    ext = "so"
    
lib = cdll.LoadLibrary('target/release/liblonlat_bng.' + ext)

Define the `ctypes` structures for lon, lat --> BNG conversion

In [3]:
class BNG_FFITuple(Structure):
    _fields_ = [("a", c_uint32),
                ("b", c_uint32)]

class BNG_FFIArray(Structure):
    _fields_ = [("data", c_void_p),
                ("len", c_size_t)]

    # Allow implicit conversions from a sequence of 32-bit unsigned
    # integers.
    @classmethod
    def from_param(cls, seq):
        return seq if isinstance(seq, cls) else cls(seq)

    # Wrap sequence of values. You can specify another type besides a
    # 32-bit unsigned integer.
    def __init__(self, seq, data_type = c_float):
        array_type = data_type * len(seq)
        raw_seq = array_type(*seq)
        self.data = cast(raw_seq, c_void_p)
        self.len = len(seq)

# A conversion function that cleans up the result value to make it
# nicer to consume.
def bng_void_array_to_tuple_list(array, _func, _args):
    res = cast(array.data, POINTER(BNG_FFITuple * array.len))[0]
    drop_array(array)
    return res

Define the `ctypes` structures for BNG --> lon, lat conversion

In [4]:
class LONLAT_FFITuple(Structure):
    _fields_ = [("a", c_float),
                ("b", c_float)]

class LONLAT_FFIArray(Structure):
    _fields_ = [("data", c_void_p),
                ("len", c_size_t)]

    # Allow implicit conversions from a sequence of 32-bit unsigned
    # integers.
    @classmethod
    def from_param(cls, seq):
        return seq if isinstance(seq, cls) else cls(seq)

    # Wrap sequence of values. You can specify another type besides a
    # 32-bit unsigned integer.
    def __init__(self, seq, data_type = c_uint32):
        array_type = data_type * len(seq)
        raw_seq = array_type(*seq)
        self.data = cast(raw_seq, c_void_p)
        self.len = len(seq)

# A conversion function that cleans up the result value to make it
# nicer to consume.
def lonlat_void_array_to_tuple_list(array, _func, _args):
    res = cast(array.data, POINTER(LONLAT_FFITuple * array.len))[0]
    return res    

Define `ctypes` input and return parameters

In [5]:
# Single-threaded
convert_vec = lib.convert_vec_c
convert_vec.argtypes = (BNG_FFIArray, BNG_FFIArray)
convert_vec.restype = BNG_FFIArray
convert_vec.errcheck = bng_void_array_to_tuple_list

# Multi-threaded
convert_bng = lib.convert_to_bng
convert_bng.argtypes = (BNG_FFIArray, BNG_FFIArray)
convert_bng.restype = BNG_FFIArray
convert_bng.errcheck = bng_void_array_to_tuple_list

convert_lonlat = lib.convert_to_lonlat
convert_lonlat.argtypes = (LONLAT_FFIArray, LONLAT_FFIArray)
convert_lonlat.restype = LONLAT_FFIArray
convert_lonlat.errcheck = lonlat_void_array_to_tuple_list

# cleanup
drop_array = lib.drop_array 
drop_array.argtypes = (POINTER(BNG_FFIArray),)

In [6]:
def convertbng(lons, lats):
    """ Multi-threaded wrapper """
    return [(i.a, i.b) for i in iter(convert_vec(lons, lats))]

def convertbng_threaded(lons, lats):
    """ Multi-threaded lon lat to BNG wrapper """
    return [(i.a, i.b) for i in iter(convert_bng(lons, lats))]

def convertlonlat_threaded(eastings, northings):
    """ Multi-threaded BNG to lon, lat wrapper """
    return [(i.a, i.b) for i in iter(convert_lonlat(eastings, northings))]

## Simple test of average conversion speed, Python version

In [7]:
%%timeit -r 10
bng(51.44533267, -0.32824866)

10000 loops, best of 10: 28.4 µs per loop


In [9]:
%%timeit -r 10
convertbng_list([-0.32824866], [51.44533267])

NameError: global name 'convertbng_list' is not defined

The threaded version's slower. Threads have overhead!

## A slightly more realistic test: 10MM random points within the UK

In [7]:
# UK bounding box
N = 55.811741
E = 1.768960
S = 49.871159
W = -6.379880

bng = pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')

lon_ls = list(np.random.uniform(W, E, [10000]))
lat_ls = list(np.random.uniform(S, N, [10000]))

### Pure Python

In [17]:
%%timeit
[bng(lat, lon) for lat, lon in zip(lat_ls, lon_ls)]

10 loops, best of 3: 80.2 ms per loop


### Pyproj

In [9]:
%%timeit -r 15
proj_res = zip(*pyproj.transform(wgs84, bng, lon_ls, lat_ls))

100 loops, best of 15: 6.97 ms per loop


### Single-threaded Rust

In [9]:
%%timeit -r 15
convertbng(lon_ls, lat_ls)

1 loops, best of 15: 1.64 s per loop


### Multithreaded Rust

In [ ]:
%%timeit -r 15
convertbng_threaded(lon_ls, lat_ls)

## Pyproj is now only ~65% faster, multithreaded Rust version is 5x faster than pure Python